As we didn't launch the ozonesonde before the **2019** convection case successfully, we have to compare the O3 profiles focasted by WACCM.

If the forcast O3 profile has the same trend but not the large difference detected by ozonesondes, the convective effect is shown. 

In [ ]:
import numpy as np
import xarray as xr
import pandas as pd
import proplot as plot
from datetime import timedelta

## Set basic parameters

In [ ]:
f_waccm = '../data/reanalysis/waccm/waccm-20190723.nc'
# set the region same as that comp_wrf_radar.ipynb
extend = [118.5, 119.5, 31.5, 32.5]
t1 = '2019-07-23 06:00'
t2 = '2019-07-25 06:00'

## Read data

In [ ]:
ds = xr.open_dataset(f_waccm)
ds

Because there's bug in the waccm time coordinate, we have to set it manually.

In [ ]:
t = ds['date'].drop_vars('time')
sec = ds['datesec'].drop_vars('time')
o3 = ds['O3'].drop_vars('time')

# calculate datetime and assign to coords
date = pd.to_datetime(t.astype(str), format='%Y%m%d') + pd.to_timedelta(sec.values, unit='s')

## Calculate the mean O3 profile in the interested region

In [ ]:
# calculate the regional mean value
mask = (o3.lon > extend[0]) & (o3.lon < extend[1]) & (o3.lat > extend[2]) & (o3.lat < extend[3])
o3 = o3.where(mask).mean(dim=['lat', 'lon']) * 1e9  # ppbv
#o3 = o3.where(mask).mean(dim=['lat', 'lon']) * 1e12  # pptv
o3.attrs['units'] = 'ppbv'

# assign the time coordinate
o3 = o3.assign_coords({'time': date})

# subset to two datetime
times = xr.DataArray(pd.to_datetime([t1, t2]), dims='time')
o3 = o3.sel(time=times)

## Plot O3 profiles

In [ ]:
fig, axs = plot.subplots(ncols=2, sharey=3, sharex=0)


ax = axs[0]
legend_labels = []
for t in o3.coords['time']:
    data = o3.sel(time=t)
    ts = pd.to_datetime(str(data.coords['time'].values))
    label = ts.strftime('%Y-%m-%d %H:%M (UTC)')
    legend_labels.append(label)
    m = ax.plot(data, data.coords['lev'])


handles, _ = ax.get_legend_handles_labels()
ax.legend(handles, legend_labels, loc='lr', ncols=1)

ax = axs[1]
diff = o3.diff('time').isel(time=0)/o3.isel(time=0) * 1e2
ax.plot(diff, diff.coords['lev'], color='k')

axs[0].format(ylabel='Pressure (hPa)',
              xlabel='O$_3$ (ppbv)',
              xlim=(0, 300),
              )

axs[1].format(ylabel='Pressure (hPa)',
              xlabel='%$\Delta$  O$_3$',
              )
axs.format(ylim=(800, 100),
           abc=True,
           abcloc='ul',
           abcstyle='(a)',
           suptitle='WACCM Chemical Forcast',
           )

fig.savefig('../figures/waccm_profiles.pdf')